In [1]:
NB_SIM = 5

In [2]:
import sys

sys.path.append('..')

In [3]:
def rec_dict_print(a_dict, level=1):
    keys = a_dict.keys()
    keys.sort()
    for k in keys:
        if type(a_dict[k]) is dict:
            print "".join(["-" for i in range(level)]) + k
            rec_dict_print(a_dict[k], level + 2)
        else:
            print "".join(["-" for i in range(level)]) + k

In [4]:
%matplotlib inline
import dill

loaded_data = None

with open('precomputed_sims/data%i.pkl' % (NB_SIM), 'rb') as f:
    loaded_data = dill.load(f)

In [5]:
rec_dict_print(loaded_data)

-N
-credit
---bc_ids
---bc_subsets_indexes
---copula
---default_times
-randomization
---distrib
-----loc
-----scale
---maturity
---zetas
-underlyings


In [6]:
N = loaded_data["N"]

In [7]:
# Instead of taking 365 standard days or 252 trading days
# in order to get some easy computations for the eqty and df time grids
# I chose to take 360 days of tradings

step = 1 / 360.
delta = 5 * step

maturity = loaded_data["randomization"]["maturity"]

print "Maturity = %s years" % maturity

Maturity = 5.0 years


# Discount

In [8]:
from finance.discountfactor import ConstantRateDiscountFactor 

r = 0.02
discount = ConstantRateDiscountFactor(r)

# Underlying

In [9]:
udlyings = loaded_data["underlyings"]

print "Maximum number of paths: %i" % len(udlyings)

gbm0 = udlyings[0]

kappa = gbm0.drifts[0][0]
sigma = gbm0.vols[0][0]
print "kappa = %s, sigma = %s" % (kappa, sigma)

time_grid = gbm0.time

Maximum number of paths: 20000
kappa = 0.12, sigma = 0.2


# Derivative

In [10]:
derivatives_nb = 1

In [11]:
from finance.products.european.swap import (
    SwapContract,
)

swap_delta = 0.25

swap_dates = SwapContract.generate_payment_dates(0, maturity, swap_delta)
swap = SwapContract(gbm0, discount, swap_dates)

price_0 = swap.price(0., incl_next_coupon=True)

print swap
print "\nPrice swap at t=0 without 1st coupon = %s" % price_0

Swap contract of maturity T = 5 years, over S^0 with strike K = 134.306, paying at {0.00, 0.25, 0.50, 0.75, 1.00, 1.25, 1.50, 1.75, 2.00, 2.25, 2.50, 2.75, 3.00, 3.25, 3.50, 3.75, 4.00, 4.25, 4.50, 4.75, 5.00}

Price swap at t=0 without 1st coupon = 0.0


In [12]:
import numpy as np

p_fixed = 1.
strike = swap.strike

delta_times = swap.delta_time
discount_factors = [discount(t) for t in swap.pillars[1:]]

delta_beta_sum = np.dot(delta_times, discount_factors)

notional = p_fixed / (strike*delta_beta_sum)

print "Notional on the swap: %s" % notional

Notional on the swap: 0.0015687485053


# Indexes stuffs

In [13]:
copula = loaded_data["credit"]["copula"]

c_subsets_indexes = loaded_data["credit"]["bc_subsets_indexes"]

obligors_nb = len(copula.subsets[c_subsets_indexes[-1]][0])
print "Obligor numbers: %s" % obligors_nb

c_ids = [17, 9, 29, 26, 50, 4, 5, 13, 64]
c_positions = [0.69, -0.46, -0.44, -0.36, 0.34, 0.23, 0.09, -0.05, -0.04]

print "Counterparties id: %s (nb = %s)" % (c_ids, len(c_ids))

Obligor numbers: 125
Counterparties id: [17, 9, 29, 26, 50, 4, 5, 13, 64] (nb = 9)


In [14]:
positions = np.zeros(obligors_nb)
for idx, ps in zip(c_ids, c_positions):
    positions[idx] = ps

positions = positions / -positions[13]
positions = np.array(positions).reshape(positions.size, 1)

print positions.flatten()

[  0.    0.    0.    0.    4.6   1.8   0.    0.    0.   -9.2   0.    0.
   0.   -1.    0.    0.    0.   13.8   0.    0.    0.    0.    0.    0.
   0.    0.   -7.2   0.    0.   -8.8   0.    0.    0.    0.    0.    0.
   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
   0.    0.    6.8   0.    0.    0.    0.    0.    0.    0.    0.    0.
   0.    0.    0.    0.   -0.8   0.    0.    0.    0.    0.    0.    0.
   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
   0.    0.    0.    0.    0. ]


## Default times

In [15]:
default_times_mat = loaded_data["credit"]["default_times"]

In [16]:
import pandas as pd

def get_default_times_sim(c_ids_, N_, default_times_mat_, copula_, subsets_indexes_):
    matrix_def_ = np.zeros((len(c_ids_), N_))
    
    for j_, c_id_ in enumerate(c_ids_):
        cp_subsets_indexes_ = copula_.get_indexes_including(c_id_)
        c_df_times_indexes_ = [ii_ for ii_, ind_ in enumerate(subsets_indexes_) if ind_ in cp_subsets_indexes_]
        
        c_df_times_mat_ = default_times_mat_[:, c_df_times_indexes_]
        c_df_times_ = c_df_times_mat_.min(axis=1)
        
        matrix_def_[j_] = c_df_times_
        
    matrix_def_[matrix_def_==1000.] = np.nan
        
    df_default_times_ = pd.DataFrame(matrix_def_, index=np.array(c_ids_))
    return df_default_times_

In [17]:
default_times = get_default_times_sim(c_ids, N, default_times_mat, copula, c_subsets_indexes)
default_times

,0,1,2,3,4,5,6,7,8,9,...,19990,19991,19992,19993,19994,19995,19996,19997,19998,19999
17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.775000,0.872222,NaN,NaN,NaN,NaN,NaN,4.311111,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,NaN,NaN,NaN,NaN,NaN,NaN,2.377778,NaN,4.636111,NaN,...,NaN,2.394444,1.691667,NaN,2.155556,NaN,2.536111,4.311111,2.555556,NaN
26,NaN,NaN,2.411111,0.35,NaN,1.438889,0.169444,NaN,0.422222,2.433333,...,2.530556,0.919444,0.800000,NaN,NaN,NaN,0.480556,4.311111,1.875000,0.552778
50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.311111,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.244444,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.219444,NaN,...,NaN,NaN,NaN,0.55,NaN,NaN,NaN,4.311111,NaN,NaN


In [18]:
import os

default_times_path = './res/default_times/'

path = os.path.join(default_times_path, 'df_times_%i.csv' %  (NB_SIM))
default_times.to_csv(path)